# Train SMel model with PCEN data

In [1]:
%load_ext autoreload
%autoreload 2

from keras import optimizers
from keras.callbacks import CSVLogger
import numpy as np
import os
import glob
import sys
import librosa

sys.path.insert(0,'../../..')
from sed_endtoend.pcen.model import SMel_PCEN
from sed_endtoend.data_generator import DataGenerator, Scaler
from sed_endtoend.gen_mel_filters import mel_filters

from params import *

os.environ["CUDA_VISIBLE_DEVICES"]="1"

# files parameters
Nfiles = None
resume = False
load_subset = Nfiles

Using TensorFlow backend.


In [2]:
params = {'sequence_time': sequence_time, 'sequence_hop_time':sequence_hop_time,
          'label_list':label_list,'audio_hop':audio_hop, 'audio_win':audio_win,
          'n_fft':n_fft,'sr':sr,'mel_bands':mel_bands,'normalize':normalize_data, 
          'frames':frames,'get_annotations':get_annotations, 'dataset': dataset}

sequence_frames = int(np.ceil(sequence_time*sr/audio_hop))

# Datasets
labels = {}# Labels

train_files = sorted(glob.glob(os.path.join(audio_folder,'train', '*.wav')))
val_files = sorted(glob.glob(os.path.join(audio_folder,'validate', '*.wav')))

if load_subset is not None:
    train_files = train_files[:load_subset]
    val_files = val_files[:load_subset]

train_labels = {}
train_mel = {}
val_labels = {}
val_mel = {}

print('Founding scaler')
for n,id in enumerate(train_files):
    labels[id] = os.path.join(label_folder, 'train',os.path.basename(id).replace('.wav','.txt'))
for id in val_files:
    labels[id] = os.path.join(label_folder, 'validate',os.path.basename(id).replace('.wav','.txt'))

params['normalize_energy'] = False
params['convert_to_dB'] = False    
    
# Generators
print('Making training generator')
training_generator = DataGenerator(train_files, labels, **params)

params['sequence_hop_time'] = sequence_time # To calculate F1_1s

print('Making validation generator')
validation_generator = DataGenerator(val_files, labels, **params)

print('Getting data')

_,S_val,mel_val,_ = validation_generator.return_all()
_,S_train,mel_train,_ = training_generator.return_all()


for j in range(mel_train.shape[0]):
    mel_train[j] = librosa.pcen(mel_train[j],sr=sr, hop_length=audio_hop, 
                                gain=alpha2, bias=delta, power=r, time_constant=time_constant) 

for j in range(mel_val.shape[0]): 
    mel_val[j] = librosa.pcen(mel_val[j],sr=sr, hop_length=audio_hop, 
                                gain=alpha2, bias=delta, power=r, time_constant=time_constant) 

Founding scaler
Making training generator
Making validation generator
Getting data
0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
0.0 %
10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %


In [3]:
print('\nBuilding model...')

sequence_samples = int(sequence_time*sr)

model = SMel_PCEN(mel_bands,sequence_frames,audio_win,audio_hop,scaler=None)    

model.summary()

# Init filters
W = librosa.filters.mel(sr,n_fft,mel_bands,htk=True);
W = W**2 #para ser consistentes
W = [W.T]#, np.zeros(filters.shape[0])]
model.layers[1].set_weights(W)

# Init PCEN layer
T = time_constant * sr / float(audio_hop)
b = (np.sqrt(1 + 4* T**2) - 1) / (2 * T**2)

r = np.ones((mel_bands,1))*r
delta = np.ones((mel_bands,1))*delta
alpha2 = np.ones((mel_bands,1))*alpha2
model.layers[-1].set_weights([alpha2,delta,r])

b = np.ones((mel_bands,))*b
model.layers[2].set_weights([b])


opt = optimizers.Adam(lr=learning_rate)

if resume:
    print('Loading best weights and resuming...')
    weights_best_file = os.path.join(expfolder, 'weights_best.hdf5')
    model.load_weights(weights_best_file)

# Fit model
print('\nFitting model...')

if resume:
    f1s_best = resume_f1_best

csv_logger = CSVLogger(os.path.join(expfolder, 'training.log'))

model.compile(loss='mean_squared_error',optimizer=opt)

history = model.fit(x=S_train, y=mel_train, batch_size=2*batch_size, #Borrar el 10!
                            epochs=epochs, verbose=fit_verbose,
                            validation_split=0.0,
                            shuffle=True,
                            callbacks=[csv_logger],
                            validation_data=(S_val,mel_val))

model.save_weights(os.path.join(expfolder, 'weights_best.hdf5'))


W1004 08:44:37.708985 139916097947392 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.




Building model...


W1004 08:44:37.932967 139916097947392 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1004 08:44:37.977474 139916097947392 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1004 08:44:38.180617 139916097947392 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1004 08:44:38.181417 139916097947392 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.C

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 44, 513)      0                                            
__________________________________________________________________________________________________
dot_1 (DOT)                     (None, 44, 128)      65664       input_1[0][0]                    
__________________________________________________________________________________________________
rnn_1 (RNN)                     (None, 44, 128)      128         dot_1[0][0]                      
__________________________________________________________________________________________________
pcen_1 (PCEN)                   (None, 44, 128)      384         dot_1[0][0]                      
                                                                 rnn_1[0][0]                      
Total para

W1004 08:44:47.370385 139916097947392 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1004 08:44:47.479465 139916097947392 deprecation.py:323] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Fitting model...
Train on 60000 samples, validate on 20000 samples
Epoch 1/11
60000/60000 [==============================] - 18s 301us/step - loss: 3.1555e-05 - val_loss: 3.0816e-05
Epoch 2/11
60000/60000 [==============================] - 16s 272us/step - loss: 2.6706e-05 - val_loss: 2.8363e-05
Epoch 3/11
60000/60000 [==============================] - 16s 272us/step - loss: 2.3189e-05 - val_loss: 1.8878e-05
Epoch 4/11
60000/60000 [==============================] - 16s 270us/step - loss: 1.9006e-05 - val_loss: 1.9979e-05
Epoch 5/11
60000/60000 [==============================] - 16s 273us/step - loss: 1.6451e-05 - val_loss: 1.7069e-05
Epoch 6/11
60000/60000 [==============================] - 16s 273us/step - loss: 1.6118e-05 - val_loss: 1.4552e-05
Epoch 7/11
60000/60000 [==============================] - 16s 273us/step - loss: 1.6003e-05 - val_loss: 1.8552e-05
Epoch 8/11
60000/60000 [==============================] - 16s 272us/step - loss: 1.5335e-05 - val_loss: 2.0865e-05
Epoch 9/11
6